In [1]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import mapping, MultiPoint
import os
from standardize_data import standardize_car_data
import sys
from geopy.distance import geodesic
from datetime import datetime
from shapely.geometry import LineString
import dask_geopandas as dgpd


In [ ]:
# Define the path to the SICAR folder
sicar_folder = "../data/SICAR/2025"

# Initialize an empty list to store GeoDataFrames
sicar_dataframes = []

# Iterate through each state folder in the SICAR directory
for state_folder in os.listdir(sicar_folder):
    state_path = os.path.join(sicar_folder, state_folder)
    if os.path.isdir(state_path):  # Ensure it's a directory
        # Look for shapefiles in the state folder
        for file in os.listdir(state_path):
            if file.endswith(".shp"):  # Check for shapefiles
                file_path = os.path.join(state_path, file)
                # Load the shapefile i nto a GeoDataFrame
                gdf = gpd.read_file(file_path)
                sicar_dataframes.append(gdf)

# Concatenate all GeoDataFrames into one
car_gdf_later_year = gpd.GeoDataFrame(pd.concat(sicar_dataframes, ignore_index=True))

car_gdf_later_year = standardize_car_data(car_gdf_later_year)
car_gdf_later_year = car_gdf_later_year[['cod_imovel','ind_status', 'ind_tipo','cod_estado', 'geometry']]
car_gdf_later_year = car_gdf_later_year[(car_gdf_later_year['ind_tipo'] == 'IRU') & (car_gdf_later_year['ind_status'].isin(['AT', 'PE']))]
car_gdf_later_year = car_gdf_later_year.drop_duplicates()

# remove invalid geometries
car_gdf_later_year = car_gdf_later_year[car_gdf_later_year.geometry.is_valid]

# For testing purposes, limit the number of rows to 500,000
car_gdf_later_year = car_gdf_later_year[:500000]

car_gdf_later_year

,cod_imovel,ind_status,ind_tipo,cod_estado,geometry
0,AP-1600253-EFCF2E144B764ED2A5415D65ED0DEC33,AT,IRU,AP,"POLYGON ((-50.69997 0.60007, -50.6992 0.60119,..."
1,AP-1600535-479CE6E131594CC4AE247EA06CBDCDE1,AT,IRU,AP,"POLYGON ((-51.97782 0.65532, -51.95378 0.64224..."
2,AP-1600709-9A375D88D9D646C1A6173F709ADAA8B1,AT,IRU,AP,"POLYGON ((-51.15353 1.55747, -51.14412 1.55216..."
3,AP-1600303-8E1CD95C52C7427B96B8345B11F6BD51,AT,IRU,AP,"POLYGON ((-51.42971 0.41301, -51.43057 0.41178..."
4,AP-1600303-B7BAC8D9E06847029ACCFAD835D10687,AT,IRU,AP,"POLYGON ((-51.04452 0.32683, -51.04411 0.31769..."
...,...,...,...,...,...
1405759,RO-1100940-25688D0A876144058634E426B995A3C2,PE,IRU,RO,"POLYGON ((-62.42183 -9.3811, -62.41906 -9.3785..."
1405760,RO-1100288-45AAAB50BC154B12BB33779A961A3538,PE,IRU,RO,"POLYGON ((-61.55923 -11.60997, -61.57016 -11.6..."
1405761,RO-1100130-CBA3543AF3144AB48EAF6F377A5366AB,AT,IRU,RO,"POLYGON ((-62.32919 -9.18088, -62.33029 -9.177..."
1405762,RO-1100940-9776279FFBC047018866DF201FFFEC8F,AT,IRU,RO,"POLYGON ((-62.4164 -9.37605, -62.41371 -9.3736..."


In [ ]:
car_gdf_earlier_year = gpd.read_file("../data/SICAR/2024/")
car_gdf_earlier_year = standardize_car_data(car_gdf_earlier_year)
car_gdf_earlier_year = car_gdf_earlier_year[['cod_imovel','ind_status', 'ind_tipo','cod_estado', 'geometry']]
car_gdf_earlier_year = car_gdf_earlier_year[(car_gdf_earlier_year['ind_tipo'] == 'IRU') & (car_gdf_earlier_year['ind_status'].isin(['AT', 'PE']))]
car_gdf_earlier_year = car_gdf_earlier_year.drop_duplicates()

# remove invalid geometries
car_gdf_earlier_year = car_gdf_earlier_year[car_gdf_earlier_year.geometry.is_valid]

# For testing purposes, limit the number of rows to 500,000
car_gdf_earlier_year = car_gdf_earlier_year[:500000]

car_gdf_earlier_year

/home/jpjon/CCCA_CAR_Analysis/myenv/lib/python3.12/site-packages/pyogrio/raw.py:198: RuntimeWarning: ../data/SICAR/2024/merged_car_2024_02_01_UTF8.shp contains polygon(s) with rings with invalid winding order. Autocorrecting them, but that shapefile should be corrected using ogr2ogr for example.
  return ogr_read(


,cod_imovel,ind_status,ind_tipo,cod_estado,geometry
0,AC-1200435-FA1FAFE4584F4A4286756A9DD56A280A,AT,IRU,AC,"POLYGON ((-70.45394 -9.35226, -70.45281 -9.352..."
1,AC-1200435-7966BE27C2484A55A1F2B635C710D9F1,AT,IRU,AC,"POLYGON ((-70.45223 -9.36378, -70.44982 -9.369..."
4,AC-1200435-AF08DCC29D964FBBAB5870C4C46F4288,AT,IRU,AC,"POLYGON ((-70.54442 -9.56007, -70.54881 -9.561..."
5,AC-1200435-AFACFDAA37FC4A789399BE90DE87317E,AT,IRU,AC,"POLYGON ((-69.89842 -9.07162, -69.90032 -9.067..."
6,AC-1200435-C78A60C479494D329B5502FFA8280C00,AT,IRU,AC,"POLYGON ((-70.18799 -9.05937, -70.1693 -9.0602..."
...,...,...,...,...,...
1278773,TO-1715101-FC5798CC6FD34FB283091D672A8144FE,AT,IRU,TO,"POLYGON ((-47.71948 -9.98304, -47.72005 -9.984..."
1278774,TO-1715101-FC6614F84E7845E0B0112DEC18A84CB5,AT,IRU,TO,"POLYGON ((-47.68281 -9.97774, -47.68476 -9.976..."
1278775,TO-1715101-FDB3736A73BC4195B3363285E7402D8D,AT,IRU,TO,"POLYGON ((-47.72366 -10.14515, -47.7176 -10.14..."
1278776,TO-1715101-F1D4F004BD974B65957E346E89B367E2,AT,IRU,TO,"POLYGON ((-47.7111 -10.07761, -47.72204 -10.08..."


In [4]:
# Define the path to the PRODES folder
prodes_folder = "../data/PRODES"

# Load the shapefile or GeoJSON file into a GeoDataFrame
# Replace 'filename.shp' with the actual file name in the PRODES folder
prodes_gdf = gpd.read_file(os.path.join(prodes_folder, 'prodes_amazonia_nb.gpkg'))

# Filter only necessary columns in PRODES
prodes_gdf = prodes_gdf[['uuid', 'geometry']]

prodes_gdf = prodes_gdf.to_crs(car_gdf_earlier_year.crs)

/home/jpjon/CCCA_CAR_Analysis/myenv/lib/python3.12/site-packages/pyogrio/geopandas.py:265: UserWarning: More than one layer found in 'prodes_amazonia_nb.gpkg': 'accumulated_deforestation_2007_biome' (default), 'hydrography_biome', 'no_forest_biome', 'yearly_deforestation_biome', 'yearly_deforestation_smaller_than_625ha_biome', 'yearly_deforestation_nf_biome', 'residual_biome'. Specify layer parameter to avoid this warning.
  result = read_func(


In [5]:
car_ddf = dgpd.from_geopandas(car_gdf_earlier_year, npartitions=4)
prodes_ddf = dgpd.from_geopandas(prodes_gdf, npartitions=4)

sjoin_ddf = car_ddf.sjoin(prodes_ddf, how="inner", predicate="intersects")
car_earlier_year_prodes_intersection_gdf = sjoin_ddf.compute().drop(columns=["index_right"])


In [11]:
car_earlier_year_prodes_intersection_gdf = car_earlier_year_prodes_intersection_gdf.drop_duplicates(subset=['cod_imovel'], keep="first")
car_earlier_year_prodes_intersection_gdf

,cod_imovel,ind_status,ind_tipo,cod_estado,geometry,uuid
0,AC-1200435-FA1FAFE4584F4A4286756A9DD56A280A,AT,IRU,AC,"POLYGON ((-70.45394 -9.35226, -70.45281 -9.352...",e02eeb9c-0a8d-42b9-be2d-c2df5e9e52ca
1,AC-1200435-7966BE27C2484A55A1F2B635C710D9F1,AT,IRU,AC,"POLYGON ((-70.45223 -9.36378, -70.44982 -9.369...",61aff6c2-b4f5-4641-8e9a-d8891bea1b54
9,AC-1200435-30219B25A3A04D049326EB9E52B89C58,AT,IRU,AC,"POLYGON ((-70.33965 -9.10346, -70.25089 -9.156...",0b4b56b9-3cef-4200-9379-19f11a703213
15,AC-1200435-F51BB0DAC613494E93382239C2DBFE00,AT,IRU,AC,"POLYGON ((-69.91369 -9.084, -69.91519 -9.08002...",db5c58df-bfe1-4bbd-b465-ade7a7a1f53a
16,AC-1200435-E1DB39B492E44E21BAC76FAEA1642598,AT,IRU,AC,"POLYGON ((-70.52553 -9.4251, -70.52819 -9.4211...",0dc43334-6e75-4333-9189-968fd7f405d7
...,...,...,...,...,...,...
522722,MT-5105150-BFA7AE24E6724CA49B4919057276D9B5,AT,IRU,MT,"POLYGON ((-58.9279 -11.46502, -58.92601 -11.46...",dd562168-132a-41e4-8bcb-3aff7af61ecc
522839,MT-5105150-F52119B6E99B48A9A18209E809892B42,AT,IRU,MT,"POLYGON ((-59.16562 -11.66544, -59.17497 -11.6...",763b7b2e-aa20-4ac2-893f-0725f84d5a1f
523323,MT-5105150-83F7C9186ABF4D6086BDD64FBFE2023F,AT,IRU,MT,"POLYGON ((-58.6278 -11.7948, -58.62773 -11.795...",4760c68d-621f-4b6a-9353-0a1a14c0f84b
523359,MT-5105150-0ECB3FF28FE14CA3B1C25AEA8DD00207,AT,IRU,MT,"POLYGON ((-58.41345 -11.76318, -58.41534 -11.7...",58c5a5be-6e35-4000-8211-f1761703428f


In [ ]:
# Just for this development script
earlier_year = 2024
later_year=2025

In [14]:
car_later_year_car_early_year_prodes_intersect = car_earlier_year_prodes_intersection_gdf.merge(
    car_gdf_later_year, 
    on="cod_imovel", 
    how="inner", 
    suffixes=(f"_{earlier_year}", f"_{later_year}")
)


In [15]:
# Sitaution where tehre are duplicates from early year to later year, for either because merge has two differnt status' for later year or because lateryear has two duplicates of same cod_imovel
car_later_year_car_early_year_prodes_intersect = car_later_year_car_early_year_prodes_intersect.drop_duplicates(subset=['cod_imovel'], keep="first")

In [16]:
car_later_year_car_early_year_prodes_intersect = car_later_year_car_early_year_prodes_intersect.copy()

car_later_year_car_early_year_prodes_intersect['geometry_changed'] = \
    car_later_year_car_early_year_prodes_intersect.apply(
        lambda row: not row[f'geometry_{earlier_year}'].equals(row[f'geometry_{later_year}']),
        axis=1
    )

In [17]:
# Merge PRODES geometry
car_later_year_car_early_year_prodes_intersect_with_prodes = car_later_year_car_early_year_prodes_intersect.merge(
    prodes_gdf[['uuid', 'geometry']],
    on='uuid',
    how='inner'
).rename(columns={'geometry': 'geometry_prodes'})

In [18]:
# Filter for cadastras that have changed yoy and where they have been changed to no longer intersect with prodes geometries

filtered_indexes = []

for row in car_later_year_car_early_year_prodes_intersect_with_prodes.itertuples():
    if row.geometry_changed and not row.__getattribute__(f'geometry_{later_year}').intersects(row.geometry_prodes):
        filtered_indexes.append(row.Index)

car_later_year_car_early_year_prodes_intersect_with_prodes = car_later_year_car_early_year_prodes_intersect_with_prodes.loc[filtered_indexes]      

car_later_year_car_early_year_prodes_intersect_with_prodes

,cod_imovel,ind_status_2024,ind_tipo_2024,cod_estado_2024,geometry_2024,uuid,ind_status_2025,ind_tipo_2025,cod_estado_2025,geometry_2025,geometry_changed,geometry_prodes
587,AC-1200450-AADAF3B8E522478EB5EE4C56780CA63E,AT,IRU,AC,"POLYGON ((-67.55517 -10.22261, -67.55198 -10.2...",84955b05-0d2f-4ac6-984a-4c34c0c972cc,PE,IRU,AC,"POLYGON ((-67.56066 -10.15905, -67.56011 -10.1...",True,"MULTIPOLYGON (((-67.51814 -10.04658, -67.51774..."
971,AC-1200450-3553C2F7AE7D4D7FB9700AC945FC8919,AT,IRU,AC,"POLYGON ((-67.16715 -9.91887, -67.18366 -9.924...",9c922fb9-52bb-4ca4-9fa5-0b2a1c636072,AT,IRU,AC,"POLYGON ((-67.16715 -9.91887, -67.17645 -9.922...",True,"MULTIPOLYGON (((-67.22628 -9.91719, -67.22595 ..."
1740,AC-1200450-7C7606E53E344F8B806336DB8444C972,AT,IRU,AC,"POLYGON ((-67.24247 -9.74865, -67.24135 -9.747...",84955b05-0d2f-4ac6-984a-4c34c0c972cc,AT,IRU,AC,"POLYGON ((-67.22702 -9.75368, -67.22703 -9.753...",True,"MULTIPOLYGON (((-67.51814 -10.04658, -67.51774..."
1865,AC-1200500-A86E1AD84CEC4DC59D9BBAA830279CB7,AT,IRU,AC,"POLYGON ((-68.74495 -9.52665, -68.74767 -9.527...",4363eb67-b52c-45c4-b1c3-57041c875844,AT,IRU,AC,"POLYGON ((-68.8479 -9.49327, -68.83874 -9.4968...",True,"MULTIPOLYGON (((-68.75602 -9.51827, -68.75495 ..."
1913,AC-1200500-AF7FC0BA924847099E5E14BE3F9053F3,AT,IRU,AC,"POLYGON ((-68.91725 -9.32429, -68.92061 -9.325...",46a9586c-a638-4649-a83c-e8dcd4029533,AT,IRU,AC,"POLYGON ((-68.77972 -9.71184, -68.78441 -9.711...",True,"MULTIPOLYGON (((-68.92926 -9.32835, -68.92872 ..."
...,...,...,...,...,...,...,...,...,...,...,...,...
104432,MT-5105507-F9AC8BB5353A44EF8363F0AED26538BB,AT,IRU,MT,"POLYGON ((-59.664 -15.21287, -59.66408 -15.217...",36bfdd6a-1555-46c2-8f9f-c3030bda6645,AT,IRU,MT,"POLYGON ((-59.97289 -16.07508, -59.96957 -16.0...",True,"MULTIPOLYGON (((-59.58032 -15.24743, -59.57774..."
108380,MT-5107941-8F8E1CE5D222408DB7390B1F6987629F,AT,IRU,MT,"POLYGON ((-56.04292 -11.23966, -56.04732 -11.2...",d80b8a76-705a-4104-8330-f23492f16d61,AT,IRU,MT,"POLYGON ((-56.03154 -11.20792, -56.02768 -11.2...",True,"MULTIPOLYGON (((-56.09527 -11.21055, -56.09473..."
108391,MT-5107941-9B98DBA9809549A5AEDD2FBC185923DD,AT,IRU,MT,"POLYGON ((-56.01168 -11.43451, -56.00821 -11.4...",67bc108f-2c18-401e-b39f-2309074cb9a9,AT,IRU,MT,"POLYGON ((-56.00041 -11.43437, -55.99927 -11.4...",True,"MULTIPOLYGON (((-55.96615 -11.46825, -55.96615..."
116480,MT-5105101-5C9CB23E6F554CB1AED7B0FD04DE9FC7,AT,IRU,MT,"POLYGON ((-57.31727 -10.58003, -57.31691 -10.5...",dacf8a8b-e9a6-41ca-bec1-23d145b11ed9,AT,IRU,MT,"POLYGON ((-57.32218 -10.64689, -57.32053 -10.6...",True,"MULTIPOLYGON (((-57.39474 -10.58512, -57.39454..."


In [23]:
# Function to calculate geodesic distance between centroids
def calculate_geodesic_distance(row):
    coord_earlier_year = (row[f'centroid_{earlier_year}'].y, row[f'centroid_{earlier_year}'].x)  # (lat, lon)
    coord_later_year = (row[f'centroid_{later_year}'].y, row[f'centroid_{later_year}'].x)
    return geodesic(coord_earlier_year, coord_later_year).meters

car_later_year_car_early_year_prodes_intersect_with_prodes[f'centroid_{earlier_year}'] = car_later_year_car_early_year_prodes_intersect_with_prodes[f'geometry_{earlier_year}'].centroid
car_later_year_car_early_year_prodes_intersect_with_prodes[f'centroid_{later_year}'] = car_later_year_car_early_year_prodes_intersect_with_prodes[f'geometry_{later_year}'].centroid


/tmp/ipykernel_70578/2449822746.py:7: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  car_later_year_car_early_year_prodes_intersect_with_prodes[f'centroid_{earlier_year}'] = car_later_year_car_early_year_prodes_intersect_with_prodes[f'geometry_{earlier_year}'].centroid
/tmp/ipykernel_70578/2449822746.py:8: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  car_later_year_car_early_year_prodes_intersect_with_prodes[f'centroid_{later_year}'] = car_later_year_car_early_year_prodes_intersect_with_prodes[f'geometry_{later_year}'].centroid


In [24]:
car_later_year_car_early_year_prodes_intersect_with_prodes['geodesic_distance'] = car_later_year_car_early_year_prodes_intersect_with_prodes.apply(calculate_geodesic_distance, axis=1)
car_later_year_car_early_year_prodes_intersect_with_prodes

,cod_imovel,ind_status_2024,ind_tipo_2024,cod_estado_2024,geometry_2024,uuid,ind_status_2025,ind_tipo_2025,cod_estado_2025,geometry_2025,geometry_changed,geometry_prodes,centroid_2024,centroid_2025,geodesic_distance
587,AC-1200450-AADAF3B8E522478EB5EE4C56780CA63E,AT,IRU,AC,"POLYGON ((-67.55517 -10.22261, -67.55198 -10.2...",84955b05-0d2f-4ac6-984a-4c34c0c972cc,PE,IRU,AC,"POLYGON ((-67.56066 -10.15905, -67.56011 -10.1...",True,"MULTIPOLYGON (((-67.51814 -10.04658, -67.51774...",POINT (-67.55381 -10.22771),POINT (-67.55761 -10.16322),7145.519683
971,AC-1200450-3553C2F7AE7D4D7FB9700AC945FC8919,AT,IRU,AC,"POLYGON ((-67.16715 -9.91887, -67.18366 -9.924...",9c922fb9-52bb-4ca4-9fa5-0b2a1c636072,AT,IRU,AC,"POLYGON ((-67.16715 -9.91887, -67.17645 -9.922...",True,"MULTIPOLYGON (((-67.22628 -9.91719, -67.22595 ...",POINT (-67.17664 -9.9184),POINT (-67.17245 -9.91711),481.102315
1740,AC-1200450-7C7606E53E344F8B806336DB8444C972,AT,IRU,AC,"POLYGON ((-67.24247 -9.74865, -67.24135 -9.747...",84955b05-0d2f-4ac6-984a-4c34c0c972cc,AT,IRU,AC,"POLYGON ((-67.22702 -9.75368, -67.22703 -9.753...",True,"MULTIPOLYGON (((-67.51814 -10.04658, -67.51774...",POINT (-67.23749 -9.7517),POINT (-67.22682 -9.75417),1201.643595
1865,AC-1200500-A86E1AD84CEC4DC59D9BBAA830279CB7,AT,IRU,AC,"POLYGON ((-68.74495 -9.52665, -68.74767 -9.527...",4363eb67-b52c-45c4-b1c3-57041c875844,AT,IRU,AC,"POLYGON ((-68.8479 -9.49327, -68.83874 -9.4968...",True,"MULTIPOLYGON (((-68.75602 -9.51827, -68.75495 ...",POINT (-68.8051 -9.52422),POINT (-68.8105 -9.52301),608.560661
1913,AC-1200500-AF7FC0BA924847099E5E14BE3F9053F3,AT,IRU,AC,"POLYGON ((-68.91725 -9.32429, -68.92061 -9.325...",46a9586c-a638-4649-a83c-e8dcd4029533,AT,IRU,AC,"POLYGON ((-68.77972 -9.71184, -68.78441 -9.711...",True,"MULTIPOLYGON (((-68.92926 -9.32835, -68.92872 ...",POINT (-68.92399 -9.3237),POINT (-68.783 -9.70816),45253.435538
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104432,MT-5105507-F9AC8BB5353A44EF8363F0AED26538BB,AT,IRU,MT,"POLYGON ((-59.664 -15.21287, -59.66408 -15.217...",36bfdd6a-1555-46c2-8f9f-c3030bda6645,AT,IRU,MT,"POLYGON ((-59.97289 -16.07508, -59.96957 -16.0...",True,"MULTIPOLYGON (((-59.58032 -15.24743, -59.57774...",POINT (-59.66594 -15.23941),POINT (-59.9809 -16.09897),100930.381372
108380,MT-5107941-8F8E1CE5D222408DB7390B1F6987629F,AT,IRU,MT,"POLYGON ((-56.04292 -11.23966, -56.04732 -11.2...",d80b8a76-705a-4104-8330-f23492f16d61,AT,IRU,MT,"POLYGON ((-56.03154 -11.20792, -56.02768 -11.2...",True,"MULTIPOLYGON (((-56.09527 -11.21055, -56.09473...",POINT (-56.0588 -11.22225),POINT (-56.04414 -11.22108),1606.174414
108391,MT-5107941-9B98DBA9809549A5AEDD2FBC185923DD,AT,IRU,MT,"POLYGON ((-56.01168 -11.43451, -56.00821 -11.4...",67bc108f-2c18-401e-b39f-2309074cb9a9,AT,IRU,MT,"POLYGON ((-56.00041 -11.43437, -55.99927 -11.4...",True,"MULTIPOLYGON (((-55.96615 -11.46825, -55.96615...",POINT (-55.98953 -11.47309),POINT (-55.99013 -11.47247),94.263414
116480,MT-5105101-5C9CB23E6F554CB1AED7B0FD04DE9FC7,AT,IRU,MT,"POLYGON ((-57.31727 -10.58003, -57.31691 -10.5...",dacf8a8b-e9a6-41ca-bec1-23d145b11ed9,AT,IRU,MT,"POLYGON ((-57.32218 -10.64689, -57.32053 -10.6...",True,"MULTIPOLYGON (((-57.39474 -10.58512, -57.39454...",POINT (-57.32347 -10.58638),POINT (-57.31866 -10.65418),7517.903022


In [25]:
from shapely.geometry import LineString

# can drop iru later to cut down data size

car_later_year_car_early_year_prodes_intersect_with_prodes['distance_line'] = car_later_year_car_early_year_prodes_intersect_with_prodes.apply(
    lambda row: LineString([row[f'geometry_{earlier_year}'].centroid, row[f'geometry_{later_year}'].centroid]),
    axis=1
)

car_later_year_car_early_year_prodes_intersect_with_prodes.head(1)

,cod_imovel,ind_status_2024,ind_tipo_2024,cod_estado_2024,geometry_2024,uuid,ind_status_2025,ind_tipo_2025,cod_estado_2025,geometry_2025,geometry_changed,geometry_prodes,centroid_2024,centroid_2025,geodesic_distance,distance_line
587,AC-1200450-AADAF3B8E522478EB5EE4C56780CA63E,AT,IRU,AC,"POLYGON ((-67.55517 -10.22261, -67.55198 -10.2...",84955b05-0d2f-4ac6-984a-4c34c0c972cc,PE,IRU,AC,"POLYGON ((-67.56066 -10.15905, -67.56011 -10.1...",True,"MULTIPOLYGON (((-67.51814 -10.04658, -67.51774...",POINT (-67.55381 -10.22771),POINT (-67.55761 -10.16322),7145.519683,"LINESTRING (-67.55381 -10.22771, -67.55761 -10..."


In [26]:
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Use the timestamp in the output directory
output_dir = f'./outputs/{timestamp}'
os.makedirs(output_dir, exist_ok=True)

fields_to_keep = [
    'cod_imovel', f'ind_status_{later_year}', f'cod_estado_{later_year}',
    'geodesic_distance'
]

geometry_columns = {
    f"geometry_{earlier_year}": f"geometry_{earlier_year}.geojson",
    f"geometry_{later_year}": f"geometry_{later_year}.geojson",
    f"geometry_prodes": "geometry_prodes.geojson",
    f"distance_line": "distance_lines.geojson"
}

target_crs = "EPSG:4674"

for geom_col, filename in geometry_columns.items():
    gdf_out = car_later_year_car_early_year_prodes_intersect_with_prodes[fields_to_keep + [geom_col]].copy()
    gdf_out = gdf_out.set_geometry(geom_col)
    
    # Set CRS if not already set
    if gdf_out.crs is None:
        gdf_out = gdf_out.set_crs(target_crs)
    else:
        # Reproject to EPSG:4674 if needed
        gdf_out = gdf_out.to_crs(target_crs)
    
    gdf_out.to_file(os.path.join(output_dir, filename), driver="GeoJSON")
